In [1]:
import os
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import cv2
import random
import time
from PIL import Image
import pandas as pd

In [2]:
# Library Constants
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkPoints = mp.solutions.hands.HandLandmark
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode
DrawingUtil = mp.solutions.drawing_utils

In [3]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

base_options = BaseOptions(model_asset_path='data/hand_landmarker.task')
options = HandLandmarkerOptions(base_options=base_options,
                                        num_hands=2)

detector = HandLandmarker.create_from_options(options)
def convert_detection_result(detection_result, image):
        """
        Draws all the landmarks on the hand
        Args:
            image (Image): Image to draw on
            detection_result (HandLandmarkerResult): HandLandmarker detection results
        """
        # Get a list of the landmarks
        hand_landmarks_list = detection_result.hand_landmarks
        
        for idx in range(len(hand_landmarks_list)):
            #NOTE this only works for one hand
            listofpointstoreturn = []
            hand_landmarks = hand_landmarks_list[idx]

            # Save the landmarks into a NormalizedLandmarkList
            hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
            hand_landmarks_proto.landmark.extend([
            landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
            ])
            # results = mp_hands.Hands.process(image = image)

            if hand_landmarks:
                for landmrk in hand_landmarks:
                    # for ids, landmrk in enumerate(hand_landmarks.landmark):
                        # print(ids, landmrk)
                    listofpointstoreturn.append(landmrk.x)
                    listofpointstoreturn.append(landmrk.y)
                    listofpointstoreturn.append(landmrk.z)
                return listofpointstoreturn
        
        return "Nothing Here"

I0000 00:00:1714586002.381798       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
def returndatapoints(image):
    to_detect = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
    results = detector.detect(to_detect)            
    # Draw the hand landmarks
    points = convert_detection_result(results, image)
    if points == "Nothing Here":
        border_size = 150
        frame = cv2.copyMakeBorder(image, top=border_size, 
                                bottom=border_size, left=border_size, 
                                right=border_size,
                                borderType=cv2.BORDER_CONSTANT, 
                                value=[0, 0, 0]
        )
        to_detect = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        results = detector.detect(to_detect) 
        points = convert_detection_result(results, frame)
    return points


In [5]:
def draw_landmarks_on_hand(image, detection_result):
    """
    Draws all the landmarks on the hand
    Args:
        image (Image): Image to draw on
        detection_result (HandLandmarkerResult): HandLandmarker detection results
    """
    # Get a list of the landmarks
    hand_landmarks_list = detection_result.hand_landmarks

    # Loop through the detected hands to visualize.
    for idx in range(len(hand_landmarks_list)):
        hand_landmarks = hand_landmarks_list[idx]

        # Save the landmarks into a NormalizedLandmarkList
        hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
        hand_landmarks_proto.landmark.extend([
        landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
        ])
        # Draw the landmarks on the hand
        DrawingUtil.draw_landmarks(image,
                                    hand_landmarks_proto,
                                    solutions.hands.HAND_CONNECTIONS,
                                    solutions.drawing_styles.get_default_hand_landmarks_style(),
                                    solutions.drawing_styles.get_default_hand_connections_style())


In [115]:
"""
Main game loop. Runs until the 
user presses "q".
"""    
# Create the hand detector
base_options = BaseOptions(model_asset_path='data/hand_landmarker.task')
options = HandLandmarkerOptions(base_options=base_options,
                                        num_hands=2)
detector = HandLandmarker.create_from_options(options)

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
total_data_collector = []
CURRENT_LETTER = "q"
NUM_DATA = 20


# TODO: Load video
video = cv2.VideoCapture(0)
START_TIME = time.time()
# TODO: Modify loop condition  
while video.isOpened():
    # Get the current frame
    frame = video.read()[1]

    # Convert it to an RGB image
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = cv2.flip(image, 1)
    # Convert the image to a readable format and find the hands
    to_detect = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
    results = detector.detect(to_detect)            
    

    # Draw the hand landmarks
    draw_landmarks_on_hand(image, results)
    datapoints = returndatapoints(image)
    if datapoints != "Nothing Here":
        most_recent_actual_data = datapoints

    # Change the color of the frame back
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imshow("Hand Tracking", image)

    #stops data collection at NUM_DATA
    if len(total_data_collector) >= NUM_DATA:
        break
    # Break the loop if the user presses 'q'
    if cv2.waitKey(50) & 0xFF == ord('q'):
        break
    if cv2.waitKey(50) & 0xFF == ord(CURRENT_LETTER):
        print(most_recent_actual_data)
        print(len(total_data_collector))
        if datapoints != "Nothing Here":
            total_data_collector.append(most_recent_actual_data)
            
            

video.release()
cv2.destroyAllWindows()
#cv2.imshow("Hand Tracking", image)



I0000 00:00:1714608078.279034       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1


[0.788978099822998, 0.76103675365448, 4.660839749703882e-07, 0.7460085153579712, 0.7334212064743042, -0.016669003292918205, 0.722722589969635, 0.6519158482551575, -0.016882557421922684, 0.7218729853630066, 0.5743621587753296, -0.01795915886759758, 0.7228628396987915, 0.5162820816040039, -0.018490711227059364, 0.7291969656944275, 0.6351843476295471, 0.009858917444944382, 0.7238350510597229, 0.5767468810081482, -0.0008389275171793997, 0.7262630462646484, 0.5311980247497559, -0.007358022034168243, 0.7321911454200745, 0.4925457835197449, -0.011012759059667587, 0.7581052184104919, 0.6298126578330994, 0.009846293367445469, 0.7551394104957581, 0.5567786693572998, -0.008006583899259567, 0.7568514943122864, 0.5035860538482666, -0.020186489447951317, 0.7603791356086731, 0.460390567779541, -0.02510126866400242, 0.7855048775672913, 0.6371232867240906, 0.0061346604488790035, 0.7861526012420654, 0.5786421298980713, -0.029121724888682365, 0.7793623805046082, 0.632077693939209, -0.03728536143898964, 0

In [116]:
len(total_data_collector)

1

In [117]:
#total_data_collector[2]

In [118]:
acquired_df = []
copy_of_total_data = total_data_collector
current_letter = CURRENT_LETTER
for i in range(len(copy_of_total_data)):
    copy_of_total_data[i].append(current_letter)
    acquired_df.append(copy_of_total_data[i])



In [119]:
dfcolnames = []
for i in range(1,22):
    for j in ["x","y","z"]:
        dfcolnames.append(j+str(i))
dfcolnames.append("label")

In [120]:
copy_df = acquired_df

In [121]:
#copy_df

In [122]:
# out_list = []
# for i in range(len(copy_df)):
#     out_list.append(copy_df[i][0])

In [123]:
end_df = pd.DataFrame(copy_df, columns = dfcolnames)

In [124]:
end_df

,x1,y1,z1,x2,y2,z2,x3,y3,z3,x4,...,x19,y19,z19,x20,y20,z20,x21,y21,z21,label
0,0.762309,0.670207,-1.815616e-07,0.725874,0.643163,-0.019692,0.699515,0.595339,-0.033432,0.686045,...,0.789747,0.539017,-0.058573,0.779926,0.583546,-0.057454,0.775401,0.617619,-0.050143,k


In [125]:
end_df.to_csv('data/acquired_data/' + CURRENT_LETTER + '.csv', index=False) 
              

In [126]:
# copy_df[0]